# Sprint 5 · Webinar 18 · Sesión Práctica (Caso licores latinoamericanos)

**Duración:** 100 minutos  
**Formato:** práctica guiada, con ejercicios progresivos y un reto inicial.

Esta sesión profundiza en limpieza avanzada, filtrado, segmentación, análisis exploratorio y visualización usando el dataset `licores_latam.csv`.


## Ejercicio 0 · Reto inicial (filtrado + razonamiento)

**Objetivo:** activar pensamiento analítico desde el inicio y practicar filtrado básico.

**Instrucciones:**

1. Filtra los licores cuyo **porcentaje de alcohol (ABV) sea mayor a 35%**.  
2. Dentro del filtro anterior, selecciona únicamente los de **precio mayor a 20 USD**.  
3. Responde:
   - ¿Qué países aparecen con licores de alta graduación y precio elevado?  
   - ¿Qué hipótesis puedes plantear sobre estos productos (por ejemplo, mercado objetivo, posicionamiento, estrategia premium)?

**Pista:**
- Usa combinaciones de condiciones con `&`.  
- Recuerda que el filtrado en pandas se hace así: `df[ (condición1) & (condición2) ]`.


In [ ]:
import pandas as pd

# Cargar el dataset base
df = pd.read_csv("licores_latam_nullvalues.csv")



## Ejercicio 1 · Validación y exploración estructural del dataset

**Objetivo:** asegurar que el dataset esté correctamente cargado y comprender su estructura antes de transformarlo.

**Tareas:**
1. Mostrar las primeras filas (`.head()`) para tener una visión rápida de los datos.  
2. Validar tipos de datos y buscar inconsistencias (`.info()`).  
3. Observar estadísticas numéricas (`.describe()`).

**Pistas:**
- Si alguna columna numérica aparece como `object`, probablemente necesite conversión de tipo.  
- Revisa si `country` y `category` están escritos de forma homogénea (mayúsculas, tildes, espacios).


In [ ]:
# Vista rápida de las primeras filas
df.head()

In [ ]:
# Información estructural y estadísticas básicas
df.info()

In [ ]:
df.describe()

### Preguntas guiadas (Ejercicio 1)

- ¿Qué columnas numéricas identificas? ¿Cuáles son categóricas?  
- ¿Qué rangos observas en `price_usd` y `abv_percent`?  
- ¿Hay columnas que parezcan mal tipadas (por ejemplo, números como texto)?  
- ¿Notas algo extraño en los nombres de países o categorías?


## Ejercicio 2 · Limpieza avanzada (normalización, variables derivadas, outliers)

**Objetivo:** mejorar la calidad del dataset mediante técnicas robustas de limpieza.

### Parte A: Normalización de texto
- Unifica países y categorías en formato título (`Title Case`).  
- Elimina espacios extra al inicio o final.

### Parte B: Creación de variables derivadas
- `price_per_ml` = `price_usd` / `bottle_volume_ml`.  
- `alcohol_ratio` = `abv_percent` / `price_usd` (una noción de "alcohol relativo al precio").

### Parte C: Detección de outliers con IQR
- Usa el rango intercuartílico (IQR) para detectar valores extremos en `price_usd`.  
- Opcional: repite el proceso para `export_volume_liters`.

**Pistas:**
- Para normalizar texto: `df['col'] = df['col'].str.strip().str.title()`.  
- Para IQR: calcula Q1, Q3, luego IQR = Q3 - Q1 y define límites inferior/superior.


In [ ]:
# Normalización de texto en países y categorías
df["country"] = df["country"].str.strip().str.title()
df["category"] = df["category"].str.strip().str.title()

# Creación de variables derivadas
df["price_per_ml"] = df["price_usd"] / df["bottle_volume_ml"]
df["alcohol_ratio"] = df["abv_percent"] / df["price_usd"]

df.head()

### Sección extra Ejercicio 2 · Manejo de datos faltantes en `pandas`

En este bloque vamos a trabajar **exclusivamente con valores faltantes** (`NaN`) usando el dataset `licores_latam.csv`.

**Objetivos específicos**:

1. Detectar y cuantificar valores faltantes en el dataset.
2. Aplicar estrategias clásicas de tratamiento: **eliminación** y **reemplazo**.
3. Explorar una técnica básica de **extrapolación / imputación avanzada** usando `interpolate()`.

> Recomendación didáctica: esta sección puede trabajarse en parejas, comparando qué decisiones de limpieza toma cada equipo.


### Parte A: Normalización de texto
- Unifica países y categorías en formato título (`Title Case`).  
- Elimina espacios extra al inicio o final.

### Parte B: Creación de variables derivadas
- `price_per_ml` = `price_usd` / `bottle_volume_ml`.  
- `alcohol_ratio` = `abv_percent` / `price_usd` (una noción de "alcohol relativo al precio").

### Parte C: Manejo de valores faltantes

In [ ]:
# 1. Detección y cuantificación de valores faltantes

# Conteo de valores faltantes por columna
missing_counts = df.isna().sum()
print("Valores faltantes por columna:\n", missing_counts)

# Porcentaje de datos faltantes por columna
missing_pct = df.isna().mean() * 100
print("\nPorcentaje de valores faltantes (%):\n", missing_pct.round(2))

# Tip: también puedes usar df.info() para ver valores no nulos
print("\nResumen estructural con info():")
df.info()

In [ ]:
# 2. Estrategias básicas: eliminación y reemplazo

# 2.1. Eliminación de filas con muchos nulos
# Ejemplo: eliminar filas donde más del 50% de las columnas están vacías
threshold = int(df.shape[1] * 0.5)
df_drop_heavy_na = df.dropna(thresh=threshold)
print("Filas originales:", df.shape[0])
print("Filas después de eliminar filas con >50% NA:", df_drop_heavy_na.shape[0])

# 2.2. Reemplazo (imputación simple) para columnas numéricas
# - price_usd: imputamos con la mediana
# - consumer_rating: imputamos con la media

median_price = df_drop_heavy_na["price_usd"].median()
mean_rating = df_drop_heavy_na["consumer_rating"].mean()

df_fill_simple = df_drop_heavy_na.copy()
df_fill_simple["price_usd"] = df_fill_simple["price_usd"].fillna(median_price)
df_fill_simple["consumer_rating"] = df_fill_simple["consumer_rating"].fillna(mean_rating)

print("\nValores nulos en price_usd después de imputar:", df_fill_simple["price_usd"].isna().sum())
print("Valores nulos en consumer_rating después de imputar:", df_fill_simple["consumer_rating"].isna().sum())

In [ ]:
# 3. Extrapolación / imputación avanzada con interpolate()

# Supongamos que queremos imputar valores faltantes en abv_percent
# usando una interpolación numérica sencilla.

# Para que interpolate tenga sentido, ordenamos por país y precio

df_interp = df.sort_values(["country", "price_usd"]).copy()

# Aplicamos interpolación por grupo de país
df_interp["abv_percent_interp"] = (
    df_interp.groupby("country")["abv_percent"]
    .apply(lambda s: s.interpolate(method="linear", limit_direction="both"))
)

# Comparamos algunas filas donde había NA originalmente
mask_na_original = df["abv_percent"].isna()
print("Filas con abv_percent originalmente nulo:")
print(df.loc[mask_na_original, ["liquor_name", "country", "abv_percent"]])

print("\nMismas filas después de interpolar (columna abv_percent_interp):")
print(df_interp.loc[mask_na_original, ["liquor_name", "country", "abv_percent_interp"]])

### Parte C: Detección de outliers con IQR
- Usa el rango intercuartílico (IQR) para detectar valores extremos en `price_usd`.  
- Opcional: repite el proceso para `export_volume_liters`.

**Pistas:**
- Para normalizar texto: `df['col'] = df['col'].str.strip().str.title()`.  
- Para IQR: calcula Q1, Q3, luego IQR = Q3 - Q1 y define límites inferior/superior.

In [ ]:
# Detección de outliers en precio usando IQR
Q1 = df["price_usd"].quantile(0.25)
Q3 = df["price_usd"].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers_price = df[(df["price_usd"] < lower) | (df["price_usd"] > upper)]
outliers_price

#### Mini-reto adicional (datos faltantes)

1. Identifica qué columnas categóricas (`object`) tienen valores faltantes. Propón una estrategia de imputación adecuada para cada una, por ejemplo:
   - Usar un valor como `"Desconocido"`.
   - Imputar con la **categoría más frecuente** por país (`mode`).
2. Compara las decisiones de imputación de tu equipo con las de otro equipo. ¿Cómo podrían afectar estas decisiones a:
   - El análisis de `price_segment`.
   - La detección de outliers.
3. Escribe **2 conclusiones** sobre buenas prácticas de manejo de datos faltantes en proyectos de analítica.


### Mini-reto (Outliers)

- Clasifica cada outlier en una de estas categorías (solo como análisis conceptual):  
  - Error de captura (valor poco realista).  
  - Producto premium (precio alto justificado por marca o añejamiento).  
  - Variación normal del mercado (no tan extremo).  
- Propón qué harías con cada tipo en un proyecto real (eliminar, corregir, dejar, segmentar).


## Ejercicio 3 · Segmentación y agrupación avanzada

**Objetivo:** profundizar en el análisis agrupado y descubrir patrones de negocio.

**Tareas:**
1. Crear segmentos de precio: `económico`, `medio` y `premium`.  
2. Agrupar por `country`, `category` y `price_segment`.  
3. Calcular:
   - `avg_price`: precio promedio.  
   - `avg_abv`: graduación alcohólica promedio.  
   - `total_exports`: exportaciones totales.  
   - `avg_price_ml`: precio por ml promedio.

**Pistas:**
- Usa `pd.cut()` para crear la columna de segmento de precio.  
- Usa `.groupby([...]).agg({...})` para múltiples métricas.


In [ ]:
# Creación de segmentos de precio
bins = [0, 10, 25, 100]
labels = ["económico", "medio", "premium"]
df["price_segment"] = pd.cut(df["price_usd"], bins=bins, labels=labels)

# Agrupación avanzada
grouped = (
    df
    .groupby(["country", "category", "price_segment"])
    .agg(
        avg_price=("price_usd", "mean"),
        avg_abv=("abv_percent", "mean"),
        total_exports=("export_volume_liters", "sum"),
        avg_price_ml=("price_per_ml", "mean")
    )
    .reset_index()
)

grouped.head()

### Reto adicional (Ejercicio 3)

1. Crea una columna `abv_level` con tres niveles: `baja`, `media`, `alta` graduación.  
2. Repite el agrupamiento incluyendo `abv_level`.  
3. Identifica:
   - El país con mayor diversidad de categorías.  
   - Si los licores de alta graduación tienden a ser más caros o no.


## Ejercicio 4 · Visualización (comparación profunda entre países)

**Objetivo:** comunicar hallazgos mediante visualizaciones claras y orientadas a negocio.

### Gráficos sugeridos:
- Boxplot de `abv_percent` por país.  
- Barras de `export_volume_liters` por país.  
- Dispersión `price_usd` vs `abv_percent`.  
- Mapa de calor de correlaciones entre variables numéricas.

**Pistas:**
- Usa `seaborn.boxplot`, `seaborn.scatterplot` o `DataFrame.plot(kind='bar')`.  
- Para correlaciones: `df.corr(numeric_only=True)`.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Boxplot de ABV por país
plt.figure(figsize=(10, 4))
sns.boxplot(data=df, x="country", y="abv_percent")
plt.xticks(rotation=45)
plt.title("Distribución de ABV por país")
plt.tight_layout()
plt.show()

In [ ]:
# Barras de exportaciones por país
export_por_pais = (
    df
    .groupby("country")["export_volume_liters"]
    .sum()
    .sort_values(ascending=False)
)

plt.figure(figsize=(8, 4))
export_por_pais.plot(kind="bar")
plt.title("Volumen total de exportación de licores por país")
plt.xlabel("País")
plt.ylabel("Volumen (litros)")
plt.tight_layout()
plt.show()

In [ ]:
# Dispersión precio vs ABV
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df, x="price_usd", y="abv_percent", hue="country")
plt.title("Relación entre precio y ABV")
plt.tight_layout()
plt.show()

In [ ]:
# Mapa de calor de correlaciones
plt.figure(figsize=(6, 4))
corr = df[["abv_percent", "bottle_volume_ml", "price_usd", "export_volume_liters", "price_per_ml", "alcohol_ratio"]].corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Mapa de calor de correlaciones")
plt.tight_layout()
plt.show()

### Mini-análisis guiado (Ejercicio 4)

- ¿Qué países muestran mayor consistencia en su ABV?  
- ¿Dónde hay mayor dispersión de precios?  
- ¿Observas relación entre precio y ABV?  
- ¿Qué variables parecen más correlacionadas en el mapa de calor y cómo lo interpretarías desde el negocio?


## Ejercicio 5 · Exportación de dataset procesado

**Objetivo:** preparar un archivo limpio y enriquecido listo para dashboards o análisis posteriores.

**Tareas:**
- Seleccionar columnas relevantes (incluyendo derivadas).  
- Verificar que no haya valores inválidos o tipos incorrectos.  
- Exportar el dataset final a un nuevo archivo CSV (`licores_latam_clean.csv`).

**Pista:**
- Puedes conservar todas las columnas o seleccionar solo las necesarias, según el caso de uso.


In [ ]:
# Crear una copia final del dataset procesado
df_final = df.copy()

# Exportar a CSV limpio
df_final.to_csv("licores_latam_clean.csv", index=False)

df_final.head()

## Take aways (resumen extendido)

1. **La limpieza profunda transforma el análisis:** pequeñas inconsistencias (espacios, mayúsculas, tipos erróneos) pueden sesgar métricas clave como promedios, totales y correlaciones.  
2. **El filtrado bien diseñado responde preguntas específicas:** combinar condiciones te permite aislar segmentos de interés (por ejemplo, productos premium de alta graduación).  
3. **Las variables derivadas agregan contexto de negocio:** indicadores como `price_per_ml` o `alcohol_ratio` pueden ser más útiles que las columnas originales para decisiones de pricing y portafolio.  
4. **La segmentación revela patrones invisibles a simple vista:** agrupar por país, categoría y segmento de precio ayuda a detectar oportunidades, riesgos y diferenciales competitivos.  
5. **La visualización es una herramienta de storytelling:** cada gráfico debe responder una pregunta clara o respaldar una hipótesis, no solo adornar el análisis.  
6. **Los outliers requieren criterio, no solo fórmulas:** es clave distinguir entre errores de captura y productos premium o de nicho para no eliminar información valiosa.  
7. **Exportar un dataset limpio es un entregable en sí mismo:** un archivo cuidadosamente preparado facilita el trabajo de otras personas analistas, BI o ciencia de datos y puede reutilizarse en múltiples proyectos.


#### Solución:

In [ ]:
# Filtro de reto: ABV > 35 y precio > 20 USD
df_reto = df[(df["abv_percent"] > 35) & (df["price_usd"] > 20)]
df_reto

## Cierre
**Kahoot de repaso (5 min)**
- Realizamos limpieza avanzada: outliers, strings y nulos.
- Dejamos el dataset listo para responder preguntas de negocio complejas.

**Reflexión:**
- ¿Qué criterio usamos para decidir qué hacer con los outliers?
- ¿Qué aprendiste hoy sobre la manipulación de textos (strings) en Pandas?

**Q&A y próximos pasos.**


## Siguientes Pasos
- **Próxima sesión:** Sprint 7 - Estadística descriptiva y Git.
- **Participación:** Termina de limpiar el dataset y guárdalo como CSV.
- **Recordatorios:** Revisa el concepto de IQR (Rango Intercuartil).
